<a href="https://colab.research.google.com/github/ronenbendavid/IDC_NLP/blob/master/Asi_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3
Training a neural named entity recognition (NER) tagger 

In [48]:
import torch
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('version: {}, device: {}'.format(torch.__version__, device))

version: 1.5.1+cu101, device: cuda


In this assignment you are required to build a full training and testing pipeline for a neural sequentail tagger for named entities, using LSTM.

The dataset that you will be working on is called ReCoNLL 2003, which is a corrected version of the CoNLL 2003 dataset: https://www.clips.uantwerpen.be/conll2003/ner/

[Train data](https://drive.google.com/file/d/1hG66e_OoezzeVKho1w7ysyAx4yp0ShDz/view?usp=sharing)

[Dev data](https://drive.google.com/file/d/1EAF-VygYowU1XknZhvzMi2CID65I127L/view?usp=sharing)

[Test data](https://drive.google.com/file/d/16gug5wWnf06JdcBXQbcICOZGZypgr4Iu/view?usp=sharing)

As you can see, the annotated texts are labeled according to the IOB annotation scheme, for 3 entity types: Person, Organization, Location.

**Task 1:** Write a funtion for reading the data from a single file (of the ones that are provided above). The function recieves a filepath and then it encodes every sentence individually using a pair of lists, one list contains the words and one list contains the tags. Each list pair will be added to a general list (data), which will be returned back from the function.

In [49]:
import requests
import re

def read_data(filepath):
    data = []

    result = re.compile(".*drive.google.com/file/d/([^/]*)/.*").match(filepath)
    if result:
      filepath = 'https://docs.google.com/uc?export=download&id={}'.format(result.group(1))
    print(filepath)

    response = requests.get(filepath)
    words = []
    tags = []

    for line in response.text.split('\n'):
        if not line:
            if len(words) > 0:
                data.append((words, tags))
            words = []
            tags = []
        else:
            line = line.strip().split()
            words.append(line[0].lower())
            tags.append(line[1])

    return data

train = read_data('https://drive.google.com/file/d/1hG66e_OoezzeVKho1w7ysyAx4yp0ShDz/view?usp=sharing')
dev = read_data('https://drive.google.com/file/d/1EAF-VygYowU1XknZhvzMi2CID65I127L/view?usp=sharing')
test = read_data('https://drive.google.com/file/d/16gug5wWnf06JdcBXQbcICOZGZypgr4Iu/view?usp=sharing')


https://docs.google.com/uc?export=download&id=1hG66e_OoezzeVKho1w7ysyAx4yp0ShDz
https://docs.google.com/uc?export=download&id=1EAF-VygYowU1XknZhvzMi2CID65I127L
https://docs.google.com/uc?export=download&id=16gug5wWnf06JdcBXQbcICOZGZypgr4Iu


The following Vocab class can be served as a dictionary that maps words and tags into Ids. The UNK_TOKEN should be used for words that are not part of the training data.

In [50]:
UNK_TOKEN = 0

class Vocab:
    def __init__(self):
        self.word2id = {"__unk__": UNK_TOKEN}
        self.id2word = {UNK_TOKEN: "__unk__"}
        self.n_words = 1
        
        self.tag2id = {"O":0, "B-PER":1, "I-PER": 2, "B-LOC": 3, "I-LOC": 4, "B-ORG": 5, "I-ORG": 6}
        self.id2tag = {0:"O", 1:"B-PER", 2:"I-PER", 3:"B-LOC", 4:"I-LOC", 5:"B-ORG", 6:"I-ORG"}
        
    def index_words(self, words):
      word_indexes = [self.index_word(w) for w in words]
      return word_indexes

    def index_tags(self, tags):
      tag_indexes = [self.tag2id[t] for t in tags]
      return tag_indexes
    
    def index_word(self, w):
        if w not in self.word2id:
            self.word2id[w] = self.n_words
            self.id2word[self.n_words] = w
            self.n_words += 1
        return self.word2id[w]
            

**Task 2:** Write a function prepare_data that takes one of the [train, dev, test] and the Vocab instance, for converting each pair of (words,tags) to a pair of indexes. Each pair should be added to data_sequences, which will be returned back from the function.

In [51]:
vocab = Vocab()

def prepare_data(data, vocab):
    data_sequences = []
    for words, tags in data:
      iwords = vocab.index_words(words)
      itags = vocab.index_tags(tags)
      data_sequences.append((iwords, itags))

    return data_sequences, vocab

train_sequences, vocab = prepare_data(train, vocab)
dev_sequences, vocab = prepare_data(dev, vocab)
test_sequences, vocab = prepare_data(test, vocab)

**Task 3:** Write NERNet, a PyTorch Module for labeling words with NER tags. 

*input_size:* the size of the vocabulary

*embedding_size:* the size of the embeddings

*hidden_size:* the LSTM hidden size

*output_size:* the number tags we are predicting for

*n_layers:* the number of layers we want to use in LSTM

*directions:* could 1 or 2, indicating unidirectional or bidirectional LSTM, respectively

The input for your forward function should be a single sentence tensor.

In [52]:
class NERNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers, bidirectional, embedding_dim=None, embedding=None):
        super(NERNet, self).__init__()

        in_features = hidden_size
        if bidirectional:
            in_features = hidden_size * 2

        if embedding:
            self.embedding = embedding
        else:
            self.embedding = nn.Embedding(num_embeddings=input_size, embedding_dim=embedding_dim)

        self.lstm = nn.LSTM(input_size=self.embedding.embedding_dim, hidden_size=hidden_size, num_layers=n_layers, bidirectional=bidirectional, batch_first=True)
        self.out = nn.Linear(in_features=in_features, out_features=output_size)

    def forward(self, input_sentence):
        embedded = self.embedding(input_sentence)
        output, _ = self.lstm(embedded)

        # shape for the linear layer (batch*seq, num_tags)
        output = output.view(-1, output.shape[2])
        return self.out(output)

**Task 4:** write a training loop, which takes a model (instance of NERNet) and number of epochs to train on. The loss is always CrossEntropyLoss and the optimizer is always Adam.

In [53]:
import torch.optim as optim

def train_loop(model, n_epochs):
    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Optimizer (ADAM is a fancy version of SGD)
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    print_every = 10000
    for e in range(1, n_epochs + 1):

        e_loss = 0
        loss = 0
        for counter, (sequence, labels) in enumerate(train_sequences):

            sequence_tensor = torch.LongTensor(sequence).to(device).view(1, -1)
            labels_tensor = torch.LongTensor(labels).to(device)

            # compute model output and loss
            output_batch = model(sequence_tensor)
            sentence_loss = criterion(output_batch, labels_tensor)

            optimizer.zero_grad()
            sentence_loss.backward()

            # updating weights
            optimizer.step()

            # averaging total loss
            sentence_loss = sentence_loss.item() / len(sequence)
            loss += sentence_loss
            e_loss += sentence_loss

            if counter > 0 and counter % print_every == 0:
                loss = loss / print_every
                print('Epoch %d/%d, %d/%d, Current Loss = %.4f' % (e, n_epochs, counter, len(train_sequences), loss))
                loss = 0

        e_loss = e_loss / len(train_sequences)
        print('Epoch %d/%d, Current Loss = %.4f' % (e, n_epochs, e_loss))



**Task 5:** write an evaluation loop on a trained model, using the dev and test datasets. This function print the true positive rate (TPR), also known as Recall and the opposite to false positive rate (FPR), also known as precision, of each label seperately (7 labels in total), and for all the 6 labels (except O) together. The caption argument for the function should be served for printing, so that when you print include it as a prefix.

In [54]:
import numpy as np

def evaluate_dataset(model, dataset):
    tp = 0
    fp = 0
    fn = 0
    tn = 0
    total = 0
    for counter, (sequence, labels) in enumerate(dataset):
        sequence_tensor = torch.LongTensor(sequence).to(device).view(1, -1)
        output_labels = model(sequence_tensor)

        labels = np.array(labels)
        pred_labels = torch.argmax(output_labels, dim=1).detach().cpu().numpy()

        tp += np.sum([labels[i[0]] == pred_labels[i[0]] for i in np.argwhere(pred_labels > 0)])
        tn += np.sum([labels[i[0]] == pred_labels[i[0]] for i in np.argwhere(pred_labels == 0)])

        fp += np.sum([labels[i[0]] != pred_labels[i[0]] for i in np.argwhere(pred_labels > 0)])
        fn += np.sum([labels[i[0]] != pred_labels[i[0]] for i in np.argwhere(pred_labels == 0)])

        total += len(pred_labels)

    # Precision = True Positive / (True Positive + False Positive)
    precision = tp / (tp + fp)

    # Recall = True Positive / (True Positive + False Negative)
    recall = tp / (tp + fn)

    # Accuracy = (True Positive + True Nagative) / Total
    accuracy = (tp + tn) / total

    print("tp: {}, tn: {}, fp: {}, fn: {}, precision: {}, recall: {}, accuracy: {}".format(tp, tn, fp, fn, precision, recall, accuracy))

    return recall, precision


def evaluate(model, caption):
  dev_recall, dev_precision = evaluate_dataset(model, dev_sequences)
  test_recall, test_precision = evaluate_dataset(model, test_sequences)

  # TODO - your code goes here
  print("{} - dev precision: {}, dev recall: {}, test precision: {}, test recall: {}".format(caption, dev_precision, dev_recall, test_precision, test_recall))


**Task 6:** Train and evaluate a few models, all with embedding_size=300, and with the following hyper parameters (you may use that as captions for the models as well):

Model 1: (hidden_size: 500, n_layers: 1, directions: 1)

Model 2: (hidden_size: 500, n_layers: 2, directions: 1)

Model 3: (hidden_size: 500, n_layers: 3, directions: 1)

Model 4: (hidden_size: 500, n_layers: 1, directions: 2)

Model 5: (hidden_size: 500, n_layers: 2, directions: 2)

Model 6: (hidden_size: 500, n_layers: 3, directions: 2)

Model 4: (hidden_size: 800, n_layers: 1, directions: 2)

Model 5: (hidden_size: 800, n_layers: 2, directions: 2)

Model 6: (hidden_size: 800, n_layers: 3, directions: 2)

In [55]:
def train_and_evaluate(hidden_size, n_layers, bidirectional, embedding=None, embedding_dim=300, n_epochs=10):
    model = NERNet(vocab.n_words, hidden_size, len(vocab.tag2id), n_layers, bidirectional, embedding=embedding, embedding_dim=embedding_dim).to(device)
    train_loop(model, n_epochs)

    caption = "hidden_size: {}, n_layers: {}, bidirectional: {}".format(hidden_size, n_layers, bidirectional)
    evaluate(model, caption)


In [56]:
# train_and_evaluate(500, 1, False, n_epochs=10)
# train_and_evaluate(500, 2, False, n_epochs=10)
train_and_evaluate(500, 3, False, n_epochs=10)
# train_and_evaluate(500, 1, True, n_epochs=10)
# train_and_evaluate(500, 2, True, n_epochs=10)
train_and_evaluate(500, 3, True, n_epochs=10)
# train_and_evaluate(800, 1, True, n_epochs=10)
# train_and_evaluate(800, 2, True, n_epochs=10)
train_and_evaluate(800, 3, True, n_epochs=10)

Epoch 1/10, Current Loss = 0.1221
Epoch 2/10, Current Loss = 0.0635
Epoch 3/10, Current Loss = 0.0317
Epoch 4/10, Current Loss = 0.0147
Epoch 5/10, Current Loss = 0.0076
Epoch 6/10, Current Loss = 0.0034
Epoch 7/10, Current Loss = 0.0024
Epoch 8/10, Current Loss = 0.0022
Epoch 9/10, Current Loss = 0.0014
Epoch 10/10, Current Loss = 0.0011
tp: 539.0, tn: 2996.0, fp: 186.0, fn: 222.0, fpr: 0.743448275862069, tpr: 0.7082785808147175, acr: 0.896525488206949
tp: 1061.0, tn: 6349.0, fp: 387.0, fn: 446.0, fpr: 0.7327348066298343, tpr: 0.7040477770404777, acr: 0.8989445590197743
hidden_size: 500, n_layers: 3, bidirectional: False - dev tpr: 0.7082785808147175, dev fpr: 0.743448275862069, test tpr: 0.7040477770404777, test fpr: 0.7327348066298343
Epoch 1/10, Current Loss = 0.0928
Epoch 2/10, Current Loss = 0.0316
Epoch 3/10, Current Loss = 0.0109
Epoch 4/10, Current Loss = 0.0037
Epoch 5/10, Current Loss = 0.0020
Epoch 6/10, Current Loss = 0.0013
Epoch 7/10, Current Loss = 0.0009
Epoch 8/10, Cu

**Task 6:** Download the GloVe embeddings from https://nlp.stanford.edu/projects/glove/ (use the 300-dim vectors from glove.6B.zip). Then intialize the nn.Embedding module in your NERNet with these embeddings, so that you can start your training with pre-trained vectors. Repeat Task 6 and print the results for each model.

Note: make sure that vectors are aligned with the IDs in your Vocab, in other words, make sure that for example the word with ID 0 is the first vector in the GloVe matrix of vectors that you initialize nn.Embedding with. For a dicussion on how to do that, check it this link:
https://discuss.pytorch.org/t/can-we-use-pre-trained-word-embeddings-for-weight-initialization-in-nn-embedding/1222

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
import numpy as np

glove_path = '/content/drive/My Drive/datasets/glove'

words_found = 0
weights_matrix = np.full(vocab.n_words, None)
with open(f'{glove_path}/glove.6B.300d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]

        idx = vocab.word2id.get(word, None)
        if idx is None:
            continue

        weights_matrix[idx] = np.array(line[1:]).astype(np.float)
        words_found += 1

for i in range(vocab.n_words):
    if weights_matrix[i] is None:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(300,))

weights_matrix = np.stack(weights_matrix, axis=0)

print('words_found: {} out of: {}'.format(words_found, vocab.n_words))


words_found: 7648 out of: 8263


In [59]:
num_embeddings, embedding_dim = weights_matrix.shape
embedding = nn.Embedding(num_embeddings, embedding_dim)
embedding.weight.data.copy_(torch.from_numpy(weights_matrix))

print(embedding)

Embedding(8263, 300)


In [ ]:
# train_and_evaluate(500, 1, False, n_epochs=10, embedding=embedding)
# train_and_evaluate(500, 2, False, n_epochs=10, embedding=embedding)
train_and_evaluate(500, 3, False, n_epochs=10, embedding=embedding)
# train_and_evaluate(500, 1, True, n_epochs=10, embedding=embedding)
# train_and_evaluate(500, 2, True, n_epochs=10, embedding=embedding)
train_and_evaluate(500, 3, True, n_epochs=10, embedding=embedding)
# train_and_evaluate(800, 1, True, n_epochs=10, embedding=embedding)
# train_and_evaluate(800, 2, True, n_epochs=10, embedding=embedding)
train_and_evaluate(800, 3, True, n_epochs=10, embedding=embedding)

Epoch 1/10, Current Loss = 0.0975
Epoch 2/10, Current Loss = 0.0308
Epoch 3/10, Current Loss = 0.0133
Epoch 4/10, Current Loss = 0.0076
Epoch 5/10, Current Loss = 0.0046
Epoch 6/10, Current Loss = 0.0031
Epoch 7/10, Current Loss = 0.0020
Epoch 8/10, Current Loss = 0.0016
Epoch 9/10, Current Loss = 0.0017
Epoch 10/10, Current Loss = 0.0016
tp: 623.0, tn: 3069.0, fp: 97.0, fn: 154.0, fpr: 0.8652777777777778, tpr: 0.8018018018018018, acr: 0.9363428861273142
tp: 1217.0, tn: 6484.0, fp: 218.0, fn: 324.0, fpr: 0.8480836236933798, tpr: 0.7897469175859831, acr: 0.9342472400824943
hidden_size: 500, n_layers: 3, bidirectional: False - dev tpr: 0.8018018018018018, dev fpr: 0.8652777777777778, test tpr: 0.7897469175859831, test fpr: 0.8480836236933798
Epoch 1/10, Current Loss = 0.0571
Epoch 2/10, Current Loss = 0.0115
Epoch 3/10, Current Loss = 0.0034
Epoch 4/10, Current Loss = 0.0014
Epoch 5/10, Current Loss = 0.0009
Epoch 6/10, Current Loss = 0.0007
Epoch 7/10, Current Loss = 0.0006
Epoch 8/10, 

**Good luck!**